In [41]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from envs.humanoid import HumanoidTracking
import numpy as np
import mediapy as media
import jax.numpy as jp
import mujoco
from dm_control import mjcf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Humanoid env

In [42]:
_CMU_MOCAP_JOINTS = (
    'lfemurrz', 'lfemurry', 'lfemurrx', 'ltibiarx', 'lfootrz', 'lfootrx',
    'ltoesrx', 'rfemurrz', 'rfemurry', 'rfemurrx', 'rtibiarx', 'rfootrz',
    'rfootrx', 'rtoesrx', 'lowerbackrz', 'lowerbackry', 'lowerbackrx',
    'upperbackrz', 'upperbackry', 'upperbackrx', 'thoraxrz', 'thoraxry',
    'thoraxrx', 'lowerneckrz', 'lowerneckry', 'lowerneckrx', 'upperneckrz',
    'upperneckry', 'upperneckrx', 'headrz', 'headry', 'headrx', 'lclaviclerz',
    'lclaviclery', 'lhumerusrz', 'lhumerusry', 'lhumerusrx', 'lradiusrx',
    'lwristry', 'lhandrz', 'lhandrx', 'lfingersrx', 'lthumbrz', 'lthumbrx',
    'rclaviclerz', 'rclaviclery', 'rhumerusrz', 'rhumerusry', 'rhumerusrx',
    'rradiusrx', 'rwristry', 'rhandrz', 'rhandrx', 'rfingersrx', 'rthumbrz',
    'rthumbrx')
len(_CMU_MOCAP_JOINTS)

56

In [43]:
root = mjcf.from_path("./assets/humanoid_CMU_V2019.xml")

physics = mjcf.Physics.from_mjcf_model(root).model.ptr

physics.body_pos.shape

physics.name_bodyadr

mj_model = mujoco.MjModel.from_xml_path("./assets/humanoid_CMU_V2019.xml")

params = np.array([
    # 'world',
    # 'root',
    'lhipjoint', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rhipjoint', 'rfemur', 'rtibia', 'rfoot',
    'rtoes', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 'lclavicle', 'lhumerus',
    'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb', 'rclavicle', 'rhumerus', 'rradius', 'rwrist',
    'rhand', 'rfingers', 'rthumb'])

jp.array([mujoco.mj_name2id(mj_model, mujoco.mju_str2Type("body"), body) for body in params])

Array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], dtype=int32)

In [44]:
params = {
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 6,
    "clip_path": "clips/test_traj.p",
    "body_names":
    ['lhipjoint', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rhipjoint', 'rfemur', 'rtibia', 'rfoot',
    'rtoes', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 'lclavicle', 'lhumerus',
    'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb', 'rclavicle', 'rhumerus', 'rradius', 'rwrist',
    'rhand', 'rfingers', 'rthumb'],
    "joint_names":
    ['root', 'lowerbackrz', 'lowerbackry', 'lowerbackrx', 'upperbackrz', 'upperbackry', 'upperbackrx', 'thoraxrz',
     'thoraxry', 'thoraxrx', 'lowerneckrz', 'lowerneckry', 'lowerneckrx', 'upperneckrz', 'upperneckry', 'upperneckrx', 'headrz', 'headry',
     'headrx', 'lclaviclerz', 'lclaviclery', 'lhumerusrz', 'lhumerusry', 'lhumerusrx', 'lradiusrx', 'lwristry', 'lhandrz', 'lhandrx', 'lfingersrx',
     'lthumbrz', 'lthumbrx', 'rclaviclerz', 'rclaviclery', 'rhumerusrz', 'rhumerusry', 'rhumerusrx', 'rradiusrx', 'rwristry', 'rhandrz',
     'rhandrx', 'rfingersrx', 'rthumbrz', 'rthumbrx', 'lfemurrz', 'lfemurry', 'lfemurrx', 'ltibiarx', 'lfootrz', 'lfootrx',
     'ltoesrx', 'rfemurrz', 'rfemurry', 'rfemurrx', 'rtibiarx', 'rfootrz', 'rfootrx', 'rtoesrx']
}

In [45]:
env = HumanoidTracking(params)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitatio

In [46]:
env.sys.nv # nq=63, nv=62

62

In [35]:
lst =[]
for n in range(100):
      joint = mujoco.mj_id2name(env.sys.mj_model, 
                          mujoco.mju_str2Type("joint"), 
                          n)
      lst.append(joint)
# root joint is a free joint in any direction (adds extra dimensions), rotation locks, quaternion
# something for full range of motion

lst

['root',
 'lowerbackrz',
 'lowerbackry',
 'lowerbackrx',
 'upperbackrz',
 'upperbackry',
 'upperbackrx',
 'thoraxrz',
 'thoraxry',
 'thoraxrx',
 'lowerneckrz',
 'lowerneckry',
 'lowerneckrx',
 'upperneckrz',
 'upperneckry',
 'upperneckrx',
 'headrz',
 'headry',
 'headrx',
 'lclaviclerz',
 'lclaviclery',
 'lhumerusrz',
 'lhumerusry',
 'lhumerusrx',
 'lradiusrx',
 'lwristry',
 'lhandrz',
 'lhandrx',
 'lfingersrx',
 'lthumbrz',
 'lthumbrx',
 'rclaviclerz',
 'rclaviclery',
 'rhumerusrz',
 'rhumerusry',
 'rhumerusrx',
 'rradiusrx',
 'rwristry',
 'rhandrz',
 'rhandrx',
 'rfingersrx',
 'rthumbrz',
 'rthumbrx',
 'lfemurrz',
 'lfemurry',
 'lfemurrx',
 'ltibiarx',
 'lfootrz',
 'lfootrx',
 'ltoesrx',
 'rfemurrz',
 'rfemurry',
 'rfemurrx',
 'rtibiarx',
 'rfootrz',
 'rfootrx',
 'rtoesrx',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 No

In [47]:
state = env.reset(jax.random.PRNGKey(0))

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco

In [ ]:
key = random.PRNGKey(0)
jit_step = jax.jit(env.step)
jit_reset = jax.jit(env.reset)

In [56]:
next = jit_reset(key)
print("starting rollout")
mu = 0
sigma = .1
rollout = []
for _ in range(500):
    _, key = jax.random.split(key)
    next = jit_step(next, mu + sigma * random.normal(key, shape=(env.sys.nu,)))
    rollout.append(next)

In [ ]:
import mediapy as media
import os

os.environ["MUJOCO_GL"] = "glfw"
rollout_data = [s.pipeline_state for s in rollout]

video = env.render(rollout_data, camera='side')

media.show_video(video, fps=1.0 / env.dt)

In [ ]:
state.pipeline_state.xpos.shape # not body_position

In [ ]:
env._ref_traj.body_positions.reshape([30,30]).shape

In [ ]:
state.obs.shape

# COMIC Validation Observation

In [53]:
from dm_control import composer
from dm_control.locomotion import arenas
from dm_control.locomotion import walkers
from walker import Rat
from cmu_humanoid import CMUHumanoidPositionControlled
from dm_control.locomotion.mocap import props
from dm_control.locomotion.tasks.reference_pose import tracking
from dm_control.locomotion.tasks.reference_pose import types
from dm_control.utils import io as resources
import os
import h5py
import pickle

In [58]:
# Define the path to your motion capture data file
file_name = 'clips/test_traj.h5' #'clips/all_snips_250.h5'
current_directory = os.getcwd()
TEST_FILE_PATH = os.path.join(current_directory, file_name)

with h5py.File(TEST_FILE_PATH, 'r') as f:
    dataset_keys = tuple(f.keys())
    dataset = types.ClipCollection(ids=dataset_keys,)

# Set up the mocap tracking task
task = tracking.MultiClipMocapTracking(
    walker=CMUHumanoidPositionControlled,
    arena=arenas.Floor(),
    ref_path=resources.GetResourceFilename(TEST_FILE_PATH),
    ref_steps=(1, 2, 3, 4, 5),
    min_steps=1,
    dataset=dataset,
    reward_type='comic',
)

# Initialize the environment
env = composer.Environment(task=task)
reset = env.reset()

ValueError: Compile error raised by Mujoco; run again with --pymjcf_debug for additional debug information.
Error: free joint can only be used on top level
Object name = walker/root, id = 2, line = 51
<body name="walker/root" childclass="walker/humanoid">

In [11]:
reset.observation['walker/reference_rel_bodies_pos_global'].shape

(1, 450)

In [12]:
reset.observation['walker/reference_rel_root_pos_local'].shape

(1, 15)

In [13]:
reset.observation['walker/reference_rel_joints'].shape

(1, 280)

In [14]:
env.task._walker_features.keys()

dict_keys(['position', 'quaternion', 'joints', 'center_of_mass', 'end_effectors', 'appendages', 'body_positions', 'body_quaternions', 'velocity', 'angular_velocity', 'joints_velocity'])

In [15]:
[item for item in reset.observation]

['reference_props_pos_global',
 'reference_props_quat_global',
 'walker/actuator_activation',
 'walker/appendages_pos',
 'walker/body_height',
 'walker/end_effectors_pos',
 'walker/joints_pos',
 'walker/joints_vel',
 'walker/sensors_accelerometer',
 'walker/sensors_force',
 'walker/sensors_gyro',
 'walker/sensors_torque',
 'walker/sensors_touch',
 'walker/sensors_velocimeter',
 'walker/world_zaxis',
 'walker/reference_rel_joints',
 'walker/reference_rel_bodies_pos_global',
 'walker/reference_rel_bodies_quats',
 'walker/reference_rel_bodies_pos_local',
 'walker/reference_ego_bodies_quats',
 'walker/reference_rel_root_quat',
 'walker/reference_rel_root_pos_local',
 'walker/reference_appendages_pos',
 'walker/clip_id',
 'walker/velocimeter_control',
 'walker/gyro_control',
 'walker/joints_vel_control',
 'walker/time_in_clip']

In [49]:
check_lst = ['walker/reference_rel_bodies_pos_local',
             'walker/reference_rel_bodies_pos_global',
             'walker/reference_rel_root_pos_local',
            #  'walker/reference_rel_joints'
            ]

COMIC_lst = reset.observation[check_lst[0]]
for check in check_lst[1:]:
    COMIC_lst = np.concatenate((COMIC_lst, reset.observation[check]), axis=1)

COMIC_lst

array([[ 1.83609521e-06, -6.27444004e-02, -7.69572712e-03,
        -2.63378137e-05, -6.05108663e-02, -3.70231761e-03,
         2.42019613e-02, -3.76863600e-02, -2.56658853e-03,
         1.42292849e-02, -1.31808936e-01,  1.34426060e-01,
        -1.86256789e-05, -1.15035662e-01,  1.74620073e-01,
         1.83609521e-06, -6.27444004e-02, -7.69572712e-03,
        -2.63296359e-05, -6.04546782e-02, -3.71563682e-03,
        -2.42178868e-02, -3.77889994e-02, -2.59714279e-03,
        -1.42562065e-02, -1.31809706e-01,  1.34414917e-01,
        -1.01046306e-05, -1.15018557e-01,  1.74591384e-01,
         1.83609521e-06, -6.27444004e-02, -7.69572712e-03,
         3.72300959e-05, -6.20893538e-02, -1.15808611e-02,
         6.44780851e-05, -6.16204414e-02, -1.24338873e-02,
         6.98565038e-05, -6.31881320e-02, -1.07661645e-02,
        -5.40669312e-05, -7.50748234e-02, -5.88738077e-03,
        -1.58136921e-04, -9.39404008e-02, -8.22999989e-05,
         6.98565038e-05, -6.31881320e-02, -1.07661645e-0

In [50]:
state.obs

Array([-8.98174196e-02,  3.86989236e-01,  8.70618701e-01, -1.07166640e-01,
        3.48463655e-05, -2.37769485e-02,  1.68382540e-01, -2.82965004e-02,
        2.95138061e-02,  8.94638896e-03, -4.03526425e-03,  1.04797602e-01,
       -3.38804200e-02, -7.31605291e-03,  5.07815108e-02, -8.98174196e-02,
        3.86989236e-01,  8.70618701e-01, -1.07141323e-01,  3.48612666e-05,
       -2.38294005e-02,  1.68428212e-01,  2.83877701e-02,  2.94677913e-02,
        8.96911323e-03,  3.99056077e-03,  1.04841590e-01, -3.38883996e-02,
        7.27319717e-03,  5.08055165e-02, -6.77492321e-02,  8.62317938e-06,
       -1.78335905e-02, -2.98213754e-02, -6.14546589e-05, -2.76747942e-02,
       -4.31101769e-03, -1.10223540e-04, -3.49184275e-02,  1.39296204e-02,
       -8.59616557e-05, -4.00781631e-02,  1.85508206e-02,  1.19673321e-04,
       -4.14772034e-02,  2.11128667e-02,  1.43156853e-04, -4.17757034e-02,
        1.39296204e-02, -8.59467546e-05, -4.00781631e-02,  7.60984421e-03,
       -2.91228294e-04, -

In [51]:
state.obs.shape

(915,)

In [39]:
COMIC_lst.shape

(1, 915)

In [52]:
np.isclose(state.obs, COMIC_lst)

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
      

# File `.p` Conversion from COMIC `.h5`

In [ ]:
# Prints the h5 file hierarchy and the dataset shapes
def print_hierarchy(h5file, indent=""):
    for key, item in h5file.items():
        if isinstance(item, h5py.Group):
            print(f"{indent}{key} (Group)")
            print_hierarchy(item, indent + "  ")
        elif isinstance(item, h5py.Dataset):
            print(f"{indent}{key} (Dataset): {item.shape}")
            
filename = "clips/test_trajectories.h5"
with h5py.File(filename, "r") as f:
    print(f"Contents of {filename}:")
    print_hierarchy(f)

In [ ]:
def extract_h5_to_dict(file):
    """Recursively extract all datasets from the HDF5 group into a dictionary."""
    data_dict = {}
    for key in file.keys():
        if isinstance(file[key], h5py.Dataset):
            # Store data as a numpy array
            data_dict[key] = file[key][:]
        elif isinstance(file[key], h5py.Group):
            # Recursively process each group
            data_dict.update({f"{key}/{subkey}": subvalue for subkey, subvalue in extract_h5_to_dict(file[key]).items()})
    return data_dict

# Path to your .h5 file
file_path = 'clips/test_traj.h5'

# Open the .h5 file and extract all data into a dictionary
with h5py.File(file_path, 'r') as file:
    all_data_dict = extract_h5_to_dict(file)

In [ ]:
all_data_dict.keys()

In [ ]:
# Dictionary to store specific data following the provided structure
structured_data = {
    'angular_velocity': all_data_dict.get('cmuv2019_001/walkers/walker_0/angular_velocity', None).T,
    'appendages': all_data_dict.get('cmuv2019_001/walkers/walker_0/appendages', None).T,
    'body_positions': all_data_dict.get('cmuv2019_001/walkers/walker_0/body_positions', None).T,
    'body_quaternions': all_data_dict.get('cmuv2019_001/walkers/walker_0/body_quaternions', None).T,
    'center_of_mass': all_data_dict.get('cmuv2019_001/walkers/walker_0/center_of_mass', None).T,
    'end_effectors': all_data_dict.get('cmuv2019_001/walkers/walker_0/end_effectors', None).T,
    'joints': all_data_dict.get('cmuv2019_001/walkers/walker_0/joints', None).T,
    'joints_velocity': all_data_dict.get('cmuv2019_001/walkers/walker_0/joints_velocity', None).T,
    'markers': all_data_dict.get('cmuv2019_001/walkers/walker_0/markers', None),
    'position': all_data_dict.get('cmuv2019_001/walkers/walker_0/position', None).T,
    'quaternion': all_data_dict.get('cmuv2019_001/walkers/walker_0/quaternion', None).T,
    'scaling': all_data_dict.get('cmuv2019_001/walkers/walker_0/scaling', None),
    'velocity': all_data_dict.get('cmuv2019_001/walkers/walker_0/velocity', None).T
}

# Optionally, print the data for verification
for key, value in structured_data.items():
    print(f"{key}: {type(value)}")

In [ ]:
[np.array(structured_data[item]).shape for item in structured_data]

In [ ]:
[item for item in structured_data]

In [ ]:
import mocap_preprocess as mp
mp.save_dataclass_pickle('clips/test_traj.p', structured_data)

# Checking if shape mataches
1. for qpos stacks, `angular_velocity`, `veclocity` matches, `joints_velocity` doesn't

In [ ]:
file_path = 'clips/humanoid_traj.p'

with open(file_path, 'rb') as file:
    data = pickle.load(file)

print(type(data))
print(data.joints_velocity.shape)

In [ ]:
file_path = 'clips/test_traj.p'

with open(file_path, 'rb') as file:
    data = pickle.load(file)

print(type(data))
print(data.joints_velocity.shape)

# Our `humanoid_traj.p` to `.h5` For COMIC

In [ ]:
cat = ['angular_velocity',
 'appendages',
 'body_positions',
 'body_quaternions',
 'center_of_mass',
 'end_effectors',
 'joints',
 'joints_velocity',
 'markers',
 'position',
 'quaternion',
 'scaling',
 'velocity']

# Path to the pickle file and the desired output HDF5 file path
pickle_file_path = 'clips/humanoid_traj.p'
hdf5_file_path = 'humanoid_traj.h5'

# Load the data from the pickle file
with open(pickle_file_path, 'rb') as file:
    data = pickle.load(file)

# Create an HDF5 file and store the data
with h5py.File(hdf5_file_path, 'w') as hdf_file:
    for mocap in cat:
        # Assuming each value is an array-like object that can be directly stored in HDF5
        if isinstance(value, np.ndarray):
            hdf_file.create_dataset(cat, data=data.mocap)
        else:
            print(f"Skipping key {key} as it is not an array.")
print(f"Data from {pickle_file_path} has been successfully written to {hdf5_file_path}.")